In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import os
import cv2
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow import keras
from scipy.ndimage import rotate
import copy
import tensorflow as tf

Using TensorFlow backend.


In [0]:
#load preprocessed data
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
patients = np.load("/gdrive/My Drive/gp/offline-patient.npy")
tumors = np.load("/gdrive/My Drive/gp/offline-tumor.npy")
np.load = np_load_old

In [0]:

# input size
img_width = 256
img_hight = 256
img_channels = 1



# dice loss function
def soft_dice_loss(y_true, y_pred, epsilon=1e-5): 
  
    numerator = 2. * tf.reduce_sum(y_pred * y_true)
    denominator = tf.reduce_sum(y_pred + y_true)
    
    return 1 - (numerator / (denominator + epsilon))


# U Net
#Encoders

layer =  tf.keras.layers
inputs = layer.Input((img_width,img_hight,img_channels))


c1 = layer.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(inputs)
c1 = layer.Dropout(0.1)(c1)
c1 = layer.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c1)
p1 = layer.AveragePooling2D((2,2))(c1)


c2 = layer.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p1)
c2 = layer.Dropout(0.1)(c2)
c2 = layer.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c2)
p2 = layer.AveragePooling2D((2,2))(c2)


c3 = layer.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p2)
c3 = layer.Dropout(0.2)(c3)
c3 = layer.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c3)
p3 = layer.AveragePooling2D((2,2))(c3)


c4 = layer.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p3)
c4 = layer.Dropout(0.2)(c4)
c4 = layer.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c4)
p4 = layer.AveragePooling2D((2,2))(c4)

c5 = layer.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p4)
c5 = layer.Dropout(0.3)(c5)
c5 = layer.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c5)


#Decoders
u6 = layer.Conv2DTranspose(128, (2,2), strides=(2,2),padding='same')(c5)
u6 = layer.concatenate([u6,c4])
c6 = layer.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u6)
c6 = layer.Dropout(0.2)(c6)
c6 = layer.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c6)


u7 = layer.Conv2DTranspose(64, (2,2), strides=(2,2),padding='valid')(c6)
u7 = layer.concatenate([u7,c3])
c7 = layer.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u7)
c7 = layer.Dropout(0.2)(c7)
c7 = layer.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c7)


u8 = layer.Conv2DTranspose(32, (2,2), strides=(2,2),padding='valid')(c7)
u8 = layer.concatenate([u8,c2])
c8 = layer.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u8)
c8 = layer.Dropout(0.1)(c8)
c8 = layer.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c8)


u9 = layer.Conv2DTranspose(16, (2,2), strides=(2,2),padding='valid')(c8)
u9 = layer.concatenate([u9,c1],axis=3)
c9 = layer.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u9)
u9 = layer.Dropout(0.1)(c9)
c9 = layer.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c9)

outputs = layer.Conv2D(1,(1,1),activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=soft_dice_loss , metrics = [tf.keras.metrics.MeanIoU(num_classes=2,name='Mean_IoU')])
model.summary()


## getting data ready to the model

In [0]:
# resize data to 256*256 
patients_256 = []
tumors_256 = []
for i in range(15):
  p256 = []
  t256 = []
  for j in range(len(patients_liver[i])):
    resized1 = cv2.resize(patients[i][j], (256,256), interpolation = cv2.INTER_AREA)
    resized3 = cv2.resize(tumors[i][j], (256,256), interpolation = cv2.INTER_AREA)
    p256.append(resized1)
    t256.append(resized3)
  patients_256.append(p256)
  tumors_256.append(t256)

In [0]:
train_len = 0
test_len = 0
for i in range(12):
  train_len += len(patients_256[i])
print(train_len)

for i in range(12,15):
  test_len += len(patients_256[i])
print(test_len)

In [0]:
# stac all slices to be ready for model 
x_train = np.zeros( (train_len, img_hight, img_width, img_channels), dtype=np.float32)
y_train = np.zeros( (train_len, img_hight, img_width, img_channels), dtype=np.float32)

x_test = np.zeros( (test_len, img_hight, img_width, img_channels), dtype=np.float32)
y_test = np.zeros( (test_len, img_hight, img_width, img_channels), dtype=np.float32)

count = 0
for i in range(12):
  for j,each_slice in enumerate(patients_256[i]):
    x_train[count] = patients_256[i][j].reshape((256,256,1))
    y_train[count] = tumors_256[i][j].reshape((256,256,1))
    count+=1
print(count)


count = 0
for i in range(12,15):
  for j,each_slice in enumerate(patients_256[i]):
    x_test[count] = patients_256[i][j].reshape((256,256,1))
    y_test[count] = tumors_256[i][j].reshape((256,256,1))
    count+=1
print(count)


In [0]:
#spiit data to tarin and valid
from sklearn.model_selection import train_test_split

trainX, validX, trainY, validY = train_test_split(x_train, y_train, test_size=0.3, random_state=2020)

In [0]:
call1 = tf.keras.callbacks.ModelCheckpoint("/gdrive/My Drive/gp/model-liver-tumor.h5",verbose=1,save_best_only=True)
call2 = tf.keras.callbacks.TensorBoard(log_dir='logs')
EPOCHS = 50
BS = 64

In [0]:
# train the network
trained = model.fit( trainX, trainY, batch_size=BS ,validation_data=(validX, validY), epochs=EPOCHS, callbacks=[call1,call2])

In [0]:
loss,acc = model.evaluate(x_train,  y_train, verbose=2)

In [0]:
predict = model.predict(x_test)
predict = (predict > 0.5).astype(np.uint8)

In [0]:
# check test score
m = tf.keras.metrics.MeanIoU(num_classes=2)
m.update_state(predict, y_test)

print('Final result: ', m.result().numpy())  


In [0]:
#see  sample
sample = 109

f, ax = plt.subplots(1,3,figsize=(14, 40))


ax[0].set_title(' patient ')
ax[0].imshow(x_train[sample].reshape((256, 256)))

ax[1].set_title(' Predicted ')
ax[1].imshow(predict[sample].reshape((256, 256)))

ax[2].set_title(' True ')
ax[2].imshow(y_train[sample].reshape((256, 256)))

plt.show()